<p id="top"></p> <br>

# APPLICATION PROGRAMMING INTERFACE (API)

## Table of Contents
<br>
<li><a href="#1">Pandas is an API</a></li>
<li><a href="#2">REST APIs Basics</a></li>
<br>

An API lets two pieces of software talk to each other. Just like a function, you don’t have to know how the API works only its inputs and outputs. An essential type of API is a REST API that allows you to access resources via the internet.

In this notebook we will review the Pandas Library in the context of an API, we will also review a basic REST API

<p id="1"></p> <br>

## Pandas is an API

Pandas is actually set of software components, much of which is not even written in Python.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [2]:
# creating some data
dict_ =  {"a":[11,21,31], "b":[12,22,32]}

When you create a Pandas object with the Dataframe constructor in API lingo, this is an "instance". The data in the dictionary is passed along to the pandas API. You then use the dataframe to communicate with the API.

In [3]:
df = pd.DataFrame(dict_)

In [4]:
type(df)

pandas.core.frame.DataFrame

In [5]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


In [6]:
df.mean()

a    21.0
b    22.0
dtype: float64

<p id="2"></p> <br>

## REST APIs Basics

Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called <a href=https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01>PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

Lets start off by getting the data we need. Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.

In [7]:
cg = CoinGeckoAPI()
bitcoin_data = cg.get_coin_market_chart_by_id(id="bitcoin", vs_currency="gbp", days=30)

In [8]:
type(bitcoin_data)

dict

In [9]:
bitcoin_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [10]:
bitcoin_price_data = bitcoin_data['prices']
bitcoin_price_data[:5]

[[1624115011431, 26248.495088826716],
 [1624118460655, 26128.38021304471],
 [1624122123386, 26059.72853865234],
 [1624125714759, 25949.680465533176],
 [1624129329186, 25746.139341690872]]

Finally lets turn this data into a Pandas DataFrame.

In [11]:
data = pd.DataFrame(bitcoin_price_data, columns=["TimeStamp", "Price"])

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [12]:
data['Date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [13]:
data.head()

,TimeStamp,Price,Date
0,1624115011431,26248.495089,2021-06-19
1,1624118460655,26128.380213,2021-06-19
2,1624122123386,26059.728539,2021-06-19
3,1624125714759,25949.680466,2021-06-19
4,1624129329186,25746.139342,2021-06-19


Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [14]:
candlestick_data = data.groupby(data.Date, as_index=False).agg({"Price":["min", "max", "first", "last"]})
candlestick_data.head()

Date         Price                                          
                        min           max         first          last
0  2021-06-19  25746.139342  26280.506505  26248.495089  26106.535181
1  2021-06-20  24565.198891  26007.784030  25903.081238  25842.626029
2  2021-06-21  22665.321521  25899.924952  25899.924952  22665.321521
3  2021-06-22  20985.197170  23835.534097  22621.693959  23118.479033
4  2021-06-23  23262.052677  24895.798944  23262.052677  23954.897739

Finally we are now ready to use plotly to create our Candlestick Chart.

In [18]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['Date'],
                open=candlestick_data['Price']['first'],
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'],
                close=candlestick_data['Price']['last'])])
fig.update_layout(xaxis_rangeslider_visible=False)

fig.update_xaxes(title="Date")
fig.update_yaxes(title="GBP")

fig.show()

<p><a href="#top">This link goes to the top</a></p>